In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima as pm
from pmdarima.arima import auto_arima
from pylab import rcParams
import requests
import json
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from xgboost import XGBRegressor

In [3]:
msft = yf.Ticker("MSFT")
msft_df = msft.history(
    period="max", 
    interval="1h",
)

print(msft_df.head())

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-06-20 09:30:00-04:00  339.695801  342.079987  336.239990  336.380005   
2023-06-20 10:30:00-04:00  336.399994  337.799988  335.859985  336.940002   
2023-06-20 11:30:00-04:00  337.000000  338.565002  336.054901  338.230011   
2023-06-20 12:30:00-04:00  338.239990  339.698608  338.190002  339.095001   
2023-06-20 13:30:00-04:00  339.109985  339.320007  338.246887  339.130005   

                            Volume  Dividends  Stock Splits  
Datetime                                                     
2023-06-20 09:30:00-04:00  6650745        0.0           0.0  
2023-06-20 10:30:00-04:00  3641713        0.0           0.0  
2023-06-20 11:30:00-04:00  2611871        0.0           0.0  
2023-06-20 12:30:00-04:00  1933145        0.0           0.0  
2023-06-20 13:30:00-04:00  1812231        0.0           0.0  


## Feature Engineering

### Lagged Prices

In [4]:
lag_periods = [1, 2, 3, 4, 5, 6, 24, 48, 72]
for lag in lag_periods:
    msft_df[f'Price_lag_{lag}H'] = msft_df['Close'].shift(lag)

### Lagged Returns

In [5]:
msft_df['Hourly_return'] = msft_df['Close'].pct_change(periods=1)
msft_df['Daily_return'] = msft_df['Close'].pct_change(periods=24)
msft_df['Weekly_return'] = msft_df['Close'].pct_change(periods=168)

In [6]:
lag_returns_hourly = [1,2,3,4,5,6,]
for lag in lag_returns_hourly:
    msft_df[f'Hourly_return_lag_{lag}H'] = msft_df['Hourly_return'].shift(lag)

In [7]:
lag_returns_daily = [24,48,74]
for lag in lag_returns_daily:
    msft_df[f'Daily_return_lag_{lag}H'] = msft_df['Daily_return'].shift(lag)

In [8]:
lag_returns_weekly = [168, 336]
for lag in lag_returns_weekly:
    msft_df[f'Weekly_return_lag_{lag}H'] = msft_df['Weekly_return'].shift(lag)